# PARTE 0 | Importes Necessários

In [1]:
import kagglehub
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PowerTransformer

# PARTE 1 | Conjunto de Dados

In [2]:
# 1. Carregar Dados do Kaggle
def carregarDados(caminhoData: str) -> pd.DataFrame:
    """
    Carrega e unifica os arquivos CSV do dataset de HRV via 'uuid'.
    """

    dataframesBrutos = {}
    try:
        arquivosCsv = [f for f in os.listdir(caminhoData) if f.endswith('.csv')]
        for arquivo in arquivosCsv:
            nomeChave = arquivo.replace('.csv', '')
            dataframesBrutos[nomeChave] = pd.read_csv(os.path.join(caminhoData, arquivo))

        listaChaves = list(dataframesBrutos.keys())
        dfUnificado = dataframesBrutos[listaChaves[0]]
        for chave in listaChaves[1:]:
            dfUnificado = pd.merge(dfUnificado, dataframesBrutos[chave], on='uuid', how='inner')

        print(f"Dataset carregado! E unificado:\n    {dfUnificado.shape}")
        return dfUnificado
    except Exception as e:
        print(f"Erro no carregamento: {e}")
        return None



# 2. Seleção e Filtragem dos Dados
def selecionarPreditoresETarget(dfEntrada: pd.DataFrame):
    """
    Isola os preditores, remove classes intermediárias e elimina variáveis
    com multicolinearidade extrema (correlação > 0.95).
    """

    # 1. Filtragem: Focar nos extremos (No Stress vs Time Pressure)
    # Removemos 'interruption' para limpar a fronteira de decisão e
    # realizar uma classificação binária, apenas
    dfLimpo = dfEntrada[dfEntrada['condition'] != 'interruption'].copy()
    dfLimpo = dfLimpo.reset_index(drop=True)

    # 2. Mapeamento Binário
    mapeamentoClasses = {'no stress': 0, 'time pressure': 1}
    y = dfLimpo['condition'].map(mapeamentoClasses).values

    # 3. Identificação Automática de Preditores Numéricos
    # Removemos colunas não-preditoras e o próprio alvo
    colunasExcluir = ['uuid', 'datasetId', 'condition', 'target']
    xBruto = dfLimpo.drop(columns=[c for c in colunasExcluir if c in dfLimpo.columns])
    xBruto = xBruto.select_dtypes(include=[np.number]) # Garante apenas números

    # 4. Filtro de Multicolinearidade (Estratégia Inteligente)
    # Calculamos a matriz de correlação absoluta
    matrizCorr = xBruto.corr().abs()

    # Selecionamos o triângulo superior da matriz para identificar pares
    superior = matrizCorr.where(np.triu(np.ones(matrizCorr.shape), k=1).astype(bool))

    # Identificamos colunas com correlação acima de 0.95
    colunasParaRemover = [coluna for coluna in superior.columns if any(superior[coluna] > 0.95)]

    x = xBruto.drop(columns=colunasParaRemover)

    print(f"Filtragem concluída.\n  Preditores finais: {x.shape[1]} (Removidas {len(colunasParaRemover)} redundantes).")

    return x, y

# 2.1 Equilíbrio de Amostras (Opcional)
def aplicarSubamostragem(xDados, yAlvo):
    """
    Realiza o Undersampling aleatório da classe majoritária para equilibrar o dataset.
    Retorna X e Y balanceados (proporção 50/50).
    """

    # Identificamos os índices de cada classe
    indicesClasse0 = np.where(yAlvo == 0)[0]
    indicesClasse1 = np.where(yAlvo == 1)[0]

    # Determinamos o tamanho da menor classe (geralmente Estresse)
    n_minoria = len(indicesClasse1)

    # Sorteamos aleatoriamente a mesma quantidade na classe majoritária
    np.random.seed(27) # Para reprodutibilidade
    indicesClasse0_Reduzidos = np.random.choice(indicesClasse0, n_minoria, replace=False)

    # Combinamos os índices e extraímos os dados
    indicesFinais = np.concatenate([indicesClasse0_Reduzidos, indicesClasse1])
    np.random.shuffle(indicesFinais) # Mistura os dados

    xBalanceado = xDados.iloc[indicesFinais] if isinstance(xDados, pd.DataFrame) else xDados[indicesFinais]
    yBalanceado = yAlvo[indicesFinais]

    print(f"Subamostragem concluída: {len(yBalanceado)} amostras totais (50/50).")

    return xBalanceado, yBalanceado


# 3. Divisão dos Conjuntos de Dados
def dividirDados(xDados, yAlvo, proporcaoTeste=0.2):
    """
    Apenas separa os dados em conjuntos de treino e teste de forma estratificada.
    """

    xTreino, xTeste, yTreino, yTeste = train_test_split(
        xDados, yAlvo, test_size=proporcaoTeste, random_state=42, stratify=yAlvo
    )
    print(f"Divisão dos conjuntos concluída (Teste: {proporcaoTeste*100}%).")
    return xTreino, xTeste, yTreino, yTeste


# 4. Transformação e normalização dos dados
def aplicarTransformacoes(xTreino, xTeste):
    """
    Aplica transformações para aproximar a distribuição normal e padroniza as escalas.
    Utiliza Yeo-Johnson para corrigir assimetria e StandardScaler para Z-score.
    """

    # 1. Transformação de Potência (Para aproximar da normalidade multivariada)
    # O método Yeo-Johnson é ideal para corrigir a assimetria (skewness) das features de HRV!!
    powerTrans = PowerTransformer(method='yeo-johnson')

    # 2. Padronização (Média 0, Variância 1)
    scaler = StandardScaler()

    # Aplicamos o pipeline de transformação
    # Nota: fit() apenas no treino para evitar vazamento de informação!
    xTreinoTrans = powerTrans.fit_transform(xTreino)
    xTreinoFinal = scaler.fit_transform(xTreinoTrans)

    xTesteTrans = powerTrans.transform(xTeste)
    xTesteFinal = scaler.transform(xTesteTrans)

    print("Transformação Yeo-Johnson e padronização z-score aplicadas.")
    return xTreinoFinal, xTesteFinal

In [3]:
# >> Pipeline de Execução

# 1. Download e definição de caminhos
path = kagglehub.dataset_download("vinayakshanawad/heart-rate-prediction-to-monitor-stress-level")
caminhoDados = os.path.join(path, 'Train Data', 'Train Data Zip')

Using Colab cache for faster access to the 'heart-rate-prediction-to-monitor-stress-level' dataset.


In [4]:
# 2. Ingestão e Unificação
dfHrv = carregarDados(caminhoDados)


Dataset carregado! E unificado:
    (369289, 37)


In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

def verificarNormalidade(xOriginal, xTransformado, feature_idx=0):
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    # Antes da transformação
    sns.histplot(xOriginal.iloc[:, feature_idx], kde=True, ax=axes[0], color='red')
    axes[0].set_title(f"Original: {xOriginal.columns[feature_idx]}")

    # Depois da transformação ( Yeo-Johnson + Scaler)
    sns.histplot(xTransformado[:, feature_idx], kde=True, ax=axes[1], color='green')
    axes[1].set_title(f"Após Yeo-Johnson: {xOriginal.columns[feature_idx]}")

    plt.tight_layout()
    plt.show()

# Exemplo de uso:
# verificarNormalidade(xTreinoBruto, xTreino, feature_idx=2)

In [6]:
if dfHrv is not None:
    # 3. Seleção de Features e Mapeamento do Target (Binário)
    # Aqui a função retorna os dados brutos ainda, apenas selecionados
    xBruto, yAlvo = selecionarPreditoresETarget(dfHrv)

    # 3.1 Aplicação da Subamostragem
    xBalanceado, yBalanceado = aplicarSubamostragem(xBruto, yAlvo)

    # 4. Divisão do Conjunto de Dados (Estratificada)
    # Fundamental dividir ANTES de qualquer transformação para evitar Data Leakage!!!
    xTreinoBruto, xTesteBruto, yTreino, yTeste = dividirDados(xBalanceado, yBalanceado, proporcaoTeste=0.2)

    # 5. Transformação (Normalidade via Yeo-Johnson + Padronização Z-score)
    # Agora aplicamos a matemática para atender às premissas do LDA
    xTreino, xTeste = aplicarTransformacoes(xTreinoBruto, xTesteBruto)

    print("\nPipeline concluído com sucesso!")

Filtragem concluída.
  Preditores finais: 23 (Removidas 11 redundantes).
Subamostragem concluída: 128114 amostras totais (50/50).
Divisão dos conjuntos concluída (Teste: 20.0%).
Transformação Yeo-Johnson e padronização z-score aplicadas.

Pipeline concluído com sucesso!


# PARTE 2 | Análise Discriminante Linear

In [7]:
class MeuClassificadorLDA:
    """
    Implementação manual da Análise Discriminante Linear (LDA).
    O modelo busca encontrar um vetor que maximize a separação entre classes
    utilizando a função discriminante linear:

    δ_k(x) = xᵀ Σ⁻¹ μ_k - 1/2 μ_kᵀ Σ⁻¹ μ_k + log(π_k)
    """

    def __init__(self):
        self.mediasClasses = {}               # Vetores mu_k | Vetor de médias da classe k
        self.probabilidadesPriori = {}        # Pesos pi_k | Probabilidade a priori da classe k
        self.matrizCovarianciaComum = None    # Matriz Sigma | Matriz de covariância comum
        self.inversaCovariancia = None        # Sigma^-1 | Inversa da matriz de covariância comum
        self.classes = None

    def treinarModelo(self, xTreino, yTreino):
        """
        Estima os parâmetros estatísticos necessários para a fronteira de decisão.
        """
        nAmostras, nPreditores = xTreino.shape
        self.classes = np.unique(yTreino)

        # Inicialização da estrutura da matriz de covariância comum
        self.matrizCovarianciaComum = np.zeros((nPreditores, nPreditores))

        for classe in self.classes:
            xClasse = xTreino[yTreino == classe]

            # 1. Cálculo do centroide (média) da classe no espaço n-dimensional
            self.mediasClasses[classe] = np.mean(xClasse, axis=0)

            # 2. Probabilidade a Priori (pi_k) balanceada
            # Aqui, em vez de xClasse.shape[0] / nAmostras, usamos pesos iguais, pois
            # neutralizamos o viés da prevalência da classe majoritária no dataset,
            # forçando o discriminante a focar exclusivamente na variância fisiológica.
            self.probabilidadesPriori[classe] = 1.0 / len(self.classes)

            # 3. Acúmulo da dispersão intra-classe para compor a matriz Sigma unificada
            desvios = xClasse - self.mediasClasses[classe]
            self.matrizCovarianciaComum += np.dot(desvios.T, desvios)

        # Cálculo da média ponderada da covariância (estimativa não enviesada)
        self.matrizCovarianciaComum /= (nAmostras - len(self.classes))

        # 4. Cálculo da Pseudoinversa (Moore-Penrose) para contornar a singularidade
        # causada por preditores altamente correlacionados (multicolinearidade).
        self.inversaCovariancia = np.linalg.pinv(self.matrizCovarianciaComum)

        print(f"Treino concluído com prioris uniformes: {self.probabilidadesPriori}")

    def calcularDiscriminante(self, xAmostra):
        """
        Calcula o score de projeção para cada classe conforme a geometria do LDA.
        """
        scores = {}
        invSigma = self.inversaCovariancia

        for classe in self.classes:
            muK = self.mediasClasses[classe]
            piK = self.probabilidadesPriori[classe]

            # Produto escalar ponderado pela inversa da covariância
            termoLinear = np.dot(np.dot(xAmostra, invSigma), muK)

            # Penalização baseada na distância ao centroide da classe
            termoQuadratico = -0.5 * np.dot(np.dot(muK.T, invSigma), muK)

            # Logaritmo da probabilidade a priori (constante nesta versão balanceada)
            termoLogPrior = np.log(piK)

            scores[classe] = termoLinear + termoQuadratico + termoLogPrior

        return scores

    def predizer(self, xTeste):
        """
        Aplica o critério de máxima verossimilhança para rotular os novos dados.
        """
        previsoes = []
        for amostra in xTeste:
            scores = self.calcularDiscriminante(amostra)
            # Decisão baseada na classe que maximiza a função delta
            classeEscolhida = max(scores, key=scores.get)
            previsoes.append(classeEscolhida)

        return np.array(previsoes)

In [8]:
# 1. Instanciar o modelo manual
modeloLda = MeuClassificadorLDA()

# 2. Treinar com os dados transformados
modeloLda.treinarModelo(xTreino, yTreino)

# 3. Realizar as predições no conjunto de treino e teste
yPreditoTreino = modeloLda.predizer(xTreino)
yPreditoTeste = modeloLda.predizer(xTeste)

Treino concluído com prioris uniformes: {np.int64(0): 0.5, np.int64(1): 0.5}


# EXTRA | Avaliação de Desempenho

In [9]:
def calcularMetricas(yReal, yPrevisto):
    """
    Calcula métricas de desempenho para classificação binária e exibe
    um relatório estruturado no console.
    """
    # 1. Cálculos de Base (Matriz de Confusão)
    tp = np.sum((yReal == 1) & (yPrevisto == 1))
    tn = np.sum((yReal == 0) & (yPrevisto == 0))
    fp = np.sum((yReal == 0) & (yPrevisto == 1))
    fn = np.sum((yReal == 1) & (yPrevisto == 0))

    # 2. Cálculo das Métricas Derivadas
    acuracia = (tp + tn) / (tp + tn + fp + fn)

    # Sensibilidade (Recall): Capacidade de detectar a classe 1 (Estresse)
    sensibilidade = tp / (tp + fn) if (tp + fn) > 0 else 0

    # Especificidade: Capacidade de detectar a classe 0 (Calma)
    especificidade = tn / (tn + fp) if (tn + fp) > 0 else 0

    # Precisão: Quando o modelo diz que é estresse, qual a chance de ser verdade?
    precisao = tp / (tp + fp) if (tp + fp) > 0 else 0

    # F1-Score: Média harmônica entre precisão e sensibilidade
    f1 = 2 * (precisao * sensibilidade) / (precisao + sensibilidade) if (precisao + sensibilidade) > 0 else 0

    # 3. Prints Organizados e Estruturados)
    print("Relatório de Desempenho da LDA")
    print("="*45)

    print(f"{'Métrica':<25} | {'Valor':<15}")
    print("-" * 45)
    print(f"{'Acurácia Global':<25} | {acuracia:.2%}")
    print(f"{'Sensibilidade (Recall)':<25} | {sensibilidade:.2%}")
    print(f"{'Especificidade':<25} | {especificidade:.2%}")
    print(f"{'Precisão':<25} | {precisao:.2%}")
    print(f"{'F1-Score':<25} | {f1:.4f}")

    print("\n\n")
    print("Matriz de Confusão")
    print("-" * 45)
    print(f"{'':>18} | {'Previsto: 0':<12} | {'Previsto: 1':<12}")
    print(f"{'Real: 0 (Calma)':>18} | {tn:<12} | {fp:<12}")
    print(f"{'Real: 1 (Estresse)':>18} | {fn:<12} | {tp:<12}")
    print("="*45 + "\n")

    matrizConfusao = np.array([[tn, fp], [fn, tp]])
    return acuracia, matrizConfusao

In [10]:
# 4. Calcular métricas finais no teste
print("Treino")
acuraciaGeralTreino, matrizConfusaoTreino = calcularMetricas(yTreino, yPreditoTreino)

Treino
Relatório de Desempenho da LDA
Métrica                   | Valor          
---------------------------------------------
Acurácia Global           | 75.44%
Sensibilidade (Recall)    | 77.51%
Especificidade            | 73.37%
Precisão                  | 74.43%
F1-Score                  | 0.7594



Matriz de Confusão
---------------------------------------------
                   | Previsto: 0  | Previsto: 1 
   Real: 0 (Calma) | 37599        | 13646       
Real: 1 (Estresse) | 11523        | 39723       



In [11]:
# 4. Calcular métricas finais no teste
print("Teste")
acuraciaGeralTeste, matrizConfusaoTeste = calcularMetricas(yTeste, yPreditoTeste)

Teste
Relatório de Desempenho da LDA
Métrica                   | Valor          
---------------------------------------------
Acurácia Global           | 75.81%
Sensibilidade (Recall)    | 77.86%
Especificidade            | 73.77%
Precisão                  | 74.80%
F1-Score                  | 0.7630



Matriz de Confusão
---------------------------------------------
                   | Previsto: 0  | Previsto: 1 
   Real: 0 (Calma) | 9451         | 3361        
Real: 1 (Estresse) | 2836         | 9975        



# PARTE 3 | Análise Discriminante Quadrática

In [12]:
class MeuClassificadorQDA:
    """
    Implementação manual da Análise Discriminante Quadrática (QDA).
    A fronteira de decisão é quadrática, permitindo que cada classe tenha sua
    própria estrutura de covariância. A equação do discriminante é:

    δ_k(x) = -1/2 log|Σ_k| - 1/2 (x - μ_k)ᵀ Σ_k⁻¹ (x - μ_k) + log(π_k)
    """

    def __init__(self):
        self.mediasClasses = {}             # Vetores mu_k | Vetor de médias da classe k
        self.inversasCovariancia = {}       # Matrizes Sigma_k^-1 | Inversas das matrizes de covariância por classe
        self.determinantesLog = {}          # log|Sigma_k| | Logaritmo do determinante da matriz de covariância por classe
        self.probabilidadesPriori = {}      # Pesos pi_k | Probabilidade a priori da classe k
        self.classes = None

    def treinarModelo(self, xTreino, yTreino):
        """
        Estima médias e matrizes de covariância específicas para cada classe.
        """
        self.classes = np.unique(yTreino)

        for classe in self.classes:
            xClasse = xTreino[yTreino == classe]

            # 1. Média da classe
            self.mediasClasses[classe] = np.mean(xClasse, axis=0)

            # 2. Probabilidade a Priori (50/50 para balanceamento)
            self.probabilidadesPriori[classe] = 1.0 / len(self.classes)

            # 3. Matriz de Covariância Específica (Σ_k)
            # Calculamos a covariância apenas com os dados desta classe
            matrizCov = np.cov(xClasse, rowvar=False)

            # 4. Estabilização e Inversão
            # Usamos a Pseudoinversa e calculamos o Log do Determinante para a fórmula
            self.inversasCovariancia[classe] = np.linalg.pinv(matrizCov)

            # Determinante usando SVD para estabilidade numérica em matrizes grandes
            signo, logdet = np.linalg.slogdet(matrizCov)
            self.determinantesLog[classe] = logdet

        print(f"Treino do QDA concluído para as classes: {self.classes}")

    def calcularDiscriminante(self, xAmostra):
        """
        Calcula o score quadrático δ_k(x).
        """
        scores = {}
        for classe in self.classes:
            mu_k = self.mediasClasses[classe]
            invSigma_k = self.inversasCovariancia[classe]
            logDet_k = self.determinantesLog[classe]
            pi_k = self.probabilidadesPriori[classe]

            # Cálculo da Distância de Mahalanobis: (x - μ)ᵀ Σ⁻¹ (x - μ)
            diff = xAmostra - mu_k
            distanciaMahalanobis = np.dot(np.dot(diff.T, invSigma_k), diff)

            # Equação completa do discriminante quadrático
            # O termo -0.5 * logDet_k é o que diferencia o QDA do LDA
            score = -0.5 * logDet_k - 0.5 * distanciaMahalanobis + np.log(pi_k)

            scores[classe] = score

        return scores

    def predizer(self, xTeste):
        """
        Classifica as amostras com base no maior score quadrático.
        """
        previsoes = [max(self.calcularDiscriminante(a), key=self.calcularDiscriminante(a).get) for a in xTeste]
        return np.array(previsoes)

In [13]:
# 1. Instanciar o modelo manual do QDA
modeloQda = MeuClassificadorQDA()

# 2. Treinar com os dados transformados
modeloQda.treinarModelo(xTreino, yTreino)

# 3. Realizar as predições no conjunto de treino e teste
yPreditoQdaTreino = modeloQda.predizer(xTreino)
yPreditoQdaTeste = modeloQda.predizer(xTeste)

Treino do QDA concluído para as classes: [0 1]


# EXTRA | Avaliação de Desempenho

In [14]:
# 4. Calcular métricas finais no treino
print("Treino")
acuraciaGeralTreino, matrizConfusaoTreino = calcularMetricas(yTreino, yPreditoQdaTreino)

Treino
Relatório de Desempenho da LDA
Métrica                   | Valor          
---------------------------------------------
Acurácia Global           | 83.09%
Sensibilidade (Recall)    | 85.93%
Especificidade            | 80.25%
Precisão                  | 81.31%
F1-Score                  | 0.8356



Matriz de Confusão
---------------------------------------------
                   | Previsto: 0  | Previsto: 1 
   Real: 0 (Calma) | 41123        | 10122       
Real: 1 (Estresse) | 7209         | 44037       



In [15]:
# 4. Calcular métricas finais no teste
print("Teste")
acuraciaGeralTeste, matrizConfusaoTeste = calcularMetricas(yTeste, yPreditoQdaTeste)

Teste
Relatório de Desempenho da LDA
Métrica                   | Valor          
---------------------------------------------
Acurácia Global           | 83.10%
Sensibilidade (Recall)    | 85.72%
Especificidade            | 80.49%
Precisão                  | 81.46%
F1-Score                  | 0.8353



Matriz de Confusão
---------------------------------------------
                   | Previsto: 0  | Previsto: 1 
   Real: 0 (Calma) | 10312        | 2500        
Real: 1 (Estresse) | 1830         | 10981       

